In [1]:
import os,sys
sys.path.append(os.path.abspath(".")) 
sys.path.append(os.path.abspath("InjecAgent")) 
sys.path.append(os.path.abspath("aside"))
from aside.model import *
from aside.model_api import *
import torch
import json
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from utils.utils import *
from utils.model_utils import load_model
from utils.plot_utils import *
torch.set_grad_enabled(False)

seed_all()
from vllm import LLM, SamplingParams
from functools import partial

from agentdojo.attacks.attack_registry import ATTACKS, load_attack
from agentdojo.task_suite.load_suites import get_suite, get_suites
from agentdojo.task_suite.task_suite import *
from agentdojo.agent_pipeline.agent_pipeline import DEFENSES, AgentPipeline, PipelineConfig
from constants import *


/home/wjyeo/anaconda3/envs/ipi_hw/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/wjyeo/anaconda3/envs/ipi_hw/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/wjyeo/anaconda3/envs/ipi_hw/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/wjyeo/anaconda3/envs/ipi_hw/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/wjyeo/anaconda3/envs/ipi_hw/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/wjyeo/anaconda3/envs/ipi_hw/compiler_compat/ld: /usr/local/cuda-12.4/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/

INFO 10-14 10:08:18 [__init__.py:216] Automatically detected platform cuda.


# Load Trained Model

In [ ]:
device = 'cuda:0'
torch_dtype = torch.bfloat16
max_ctx_len = 32768
# model_path = 'Qwen/Qwen3-8B'
model_path = os.path.join(MODEL_DIR,'Qwen3-8B_sft_ASIDE_ToolAndAlpaca_LARGE')

use_vllm=True
is_api = False

model,tokenizer,is_aside,init_fn = load_model(model_path,use_vllm=use_vllm,dtype=torch_dtype,vllm_kwargs = {'gpu_memory_utilization':0.8,'enable_chunked_prefill':False},max_ctx_len=max_ctx_len) # init_fn is for aside mainly (to used on the input_ids to turn them to rotated embeddings)

    
print (f'Use Vllm: {use_vllm}, is_aside: {is_aside}')
def assign_segment_ids(texts,tokenizer,embed_fn=None):
    if isinstance(texts[0],str):
        encoded = encode_fn(encoded,tokenizer) # turn to input_ids
    segment_ids = multiturn_aside_encode(encoded,tokenizer,start_tokens = qwen_start_tokens,end_tokens = qwen_end_tokens,until_last_token = "\n<|im_start|>assistant")
    encoded['segment_ids'] = segment_ids
    input_lens = [len(tokenizer.encode(x)) for x in texts] # to unpad later
    
    # Debug cause.
    # segs = get_aside_segments(encoded[0],tokenizer)
    # pprint (f'PROMPT: {tokenizer.decode(encoded[0]["input_ids"][0])}')
    # for i, seg in enumerate(segs):
    #     pprint (f'SEG {i} {seg}')
    if embed_fn is not None:
        encoded = embed_fn(encoded,input_lens)
    return encoded

if is_aside:
    additional_encode_fn = partial(assign_segment_ids,tokenizer=tokenizer,embed_fn=init_fn)
else:
    additional_encode_fn = lambda x:x
        

`torch_dtype` is deprecated! Use `dtype` instead!



 <class 'model.CustomQwen3Config'> <class 'model.Qwen3ForwardRot'> 

CALLED load_vanilla_model_and_tokenizer on model /mnt/disk1/wjyeo/models/Qwen3-8B_sft_ASIDE_ToolAndAlpaca_LARGE and tokenizer /mnt/disk1/wjyeo/models/Qwen3-8B_sft_ASIDE_ToolAndAlpaca_LARGE
Model config CustomQwen3Config {
  "add_linear_shift": false,
  "architectures": [
    "Qwen3ForwardRot"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151645,
  "gradual_rotation": false,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 12288,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

chat_template_path: None

 MODEL TYPE:  <class 'model.Qwen3ForwardRot'>
Embedding type forward_rot
alpha_t: 1.5703125 (dtype: torch.bfloat16)
INFO 10-14 10:08:40 [utils.py:328] non-default args: {'enable_prompt_embeds': True, 'max_model_len': 32768, 'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'enable_chunked_prefill': False, 'model': '/mnt/disk1/wjyeo/models/Qwen3-8B_sft_ASIDE_ToolAndAlpaca_LARGE'}
INFO 10-14 10:08:48 [__init__.py:742] Resolved architecture: Qwen3ForwardRot


`torch_dtype` is deprecated! Use `dtype` instead!


...

alpha_t: 1.5703125 (dtype: torch.bfloat16)


Loading safetensors checkpoint shards:   0% Completed | 0/7 [00:00<?, ?it/s]


INFO 10-14 10:09:02 [default_loader.py:268] Loading weights took 9.67 seconds
INFO 10-14 10:09:03 [model_runner.py:1083] Model loading took 15.2956 GiB and 9.885095 seconds
INFO 10-14 10:09:05 [worker.py:290] Memory profiling takes 1.35 seconds
INFO 10-14 10:09:05 [worker.py:290] the current vLLM instance can use total_gpu_memory (139.81GiB) x gpu_memory_utilization (0.80) = 111.85GiB
INFO 10-14 10:09:05 [worker.py:290] model weights take 15.30GiB; non_torch_memory takes 0.07GiB; PyTorch activation peak memory takes 3.28GiB; the rest of the memory reserved for KV Cache is 93.21GiB.
INFO 10-14 10:09:05 [executor_base.py:114] # cuda blocks: 42418, # CPU blocks: 1820
INFO 10-14 10:09:05 [executor_base.py:119] Maximum concurrency for 32768 tokens per request: 20.71x
INFO 10-14 10:09:07 [model_runner.py:1355] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager

Capturing CUDA graph shapes:   0%|          | 0/70 [00:00<?, ?it/s]

INFO 10-14 10:09:41 [model_runner.py:1507] Graph capturing finished in 34 secs, took 0.67 GiB
INFO 10-14 10:09:41 [worker.py:467] Free memory on device (137.99/139.81 GiB) on startup. Desired GPU memory utilization is (0.8, 111.85 GiB). Actual usage is 15.3 GiB for weight, 3.28 GiB for peak activation, 0.07 GiB for non-torch memory, and 0.67 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=99197348659` to fit into requested memory, or `--kv-cache-memory=127264045056` to fully utilize gpu memory. Current kv cache memory in use is 100078152499 bytes.
INFO 10-14 10:09:41 [llm_engine.py:420] init engine (profile, create kv cache, warmup model) took 38.39 seconds
INFO 10-14 10:09:42 [llm.py:295] Supported_tasks: ['generate']
INFO 10-14 10:09:42 [__init__.py:36] No IOProcessor plugins requested by the model
Use Vllm: True, is_aside: True


In [10]:
def assign_segment_ids(encoded,tokenizer,embed_fn=None):
    if model.use_vllm: # perform one by one
        if not isinstance(encoded,list): # single string, make into a list
            encoded = [encoded]
        if isinstance(encoded[0],str): # encode to input_ids one by one, vllm takes unpadded strings
            encoded =[encode_fn(x,tokenizer) for x in encoded]
        segment_ids = [multiturn_aside_encode(x,tokenizer,start_tokens = qwen_start_tokens,end_tokens = qwen_end_tokens,until_last_token = "\n<|im_start|>assistant") for x in encoded] # allow the
        for e,sid in zip(encoded,segment_ids):
            e['segment_ids'] = sid
    else: # perform in batch
        if isinstance(encoded[0],str):
            encoded = encode_fn(encoded,tokenizer) # turn to input_ids
        segment_ids = multiturn_aside_encode(encoded,tokenizer,start_tokens = qwen_start_tokens,end_tokens = qwen_end_tokens,until_last_token = "\n<|im_start|>assistant")
        encoded['segment_ids'] = segment_ids
    
    # Debug cause.
    # segs = get_aside_segments(encoded[0],tokenizer)
    # pprint (f'PROMPT: {tokenizer.decode(encoded[0]["input_ids"][0])}')
    # for i, seg in enumerate(segs):
    #     pprint (f'SEG {i} {seg}')
    
    if embed_fn is not None:
        encoded = embed_fn(encoded)
    return encoded

if is_aside:
    additional_encode_fn = partial(assign_segment_ids,tokenizer=tokenizer,embed_fn=init_fn)
else:
    additional_encode_fn = lambda x:x
        

In [11]:
def tool_prompt_format(messages,tools,tokenizer,encode=False,enable_thinking=False,remove_system_prompt=False): # for tool-use agents like Qwen
    if messages[-1]['role'] == 'assistant':
        add_generation_prompt = False
    else:
        add_generation_prompt = True
    if messages[0]['role'] == 'system' and remove_system_prompt:
        messages = messages[1:] # remove system prompt for tool-use models
    if 'qwen' in tokenizer.name_or_path.lower():
        formatted = tokenizer.apply_chat_template(messages,add_generation_prompt=add_generation_prompt,tokenize=False,enable_thinking=enable_thinking,tools=tools)
    else:
        formatted = tokenizer.apply_chat_template(messages,add_generation_prompt=add_generation_prompt,tokenize=False,tools=tools)
    # pprint (formatted)
    # formatted = formatted.split('<think>')[0].strip() ## TEMP
    return encode_fn(formatted,tokenizer) if encode else formatted

In [12]:
defense = None
if not is_api:
    # Tool LLM
    agent_pipeline = AgentPipeline.from_config(
            PipelineConfig(
                llm='local-tool',
                defense=defense,
                system_message_name=None,
                system_message=None,
                tool_output_format=None,
                extra_args={
                    'model':model,
                    'tokenizer':tokenizer,
                    'encode_fn': partial(tool_prompt_format,tokenizer=tokenizer),
                    'additional_encode_fn': additional_encode_fn,
                    'gen_kwargs': {'max_new_tokens':512,'do_sample':False,'eos_token_id':[tokenizer.eos_token_id]} if not use_vllm else SamplingParams(max_tokens=512,temperature=0,stop_token_ids = [tokenizer.eos_token_id]),
                    'is_vllm': use_vllm,
                    'tool_role': 'tool' if 'metasecalign' not in model_path.lower() or 'tool' in model_path.lower() else 'input'
                },
                # max_iters=3,
            )
        )
else:
    with open(f'/export/home2/weijie210/ipi_huawei/ds_key.txt','r') as f:
        ds_key = f.read().strip()
    os.environ['OPENAI_API_KEY'] = ds_key
    agent_pipeline = AgentPipeline.from_config(
            PipelineConfig(
                llm=model_path,
                defense=defense,
                system_message_name=None,
                system_message=None,
                tool_output_format=None,
                extra_args ={'max_tokens':512}
                
            )
        )

# Function for Evaluation

In [13]:
from agentdojo.base_tasks import BaseInjectionTask, BaseUserTask

def run_task(suite,user_task,injection_task=None,task_injection = {},return_messages=False):
    environment = suite.load_and_inject_default_environment(task_injection)
    if isinstance(user_task, BaseUserTask):
        task_environment = user_task.init_environment(environment)
    else:
        task_environment = environment
    pre_environment = task_environment.model_copy(deep=True)
    if isinstance(user_task, BaseUserTask):
        prompt = user_task.PROMPT
    else:
        prompt = user_task.GOAL

    messages = []
    runtime = FunctionsRuntime(suite.tools)

    _, _, task_environment, messages, _ = agent_pipeline.query(prompt, runtime, task_environment)
    model_output = model_output_from_messages(messages)
    functions_stack_trace = functions_stack_trace_from_messages(messages)
    utility = suite._check_task_result(
            user_task,
            model_output or [],
            pre_environment,
            task_environment,  # type: ignore
            functions_stack_trace,
        )
    security = None
    if injection_task is not None:
        security = suite._check_task_result(
            injection_task,
            model_output or [],
            pre_environment,
            task_environment,  # type: ignore
            functions_stack_trace,
        )
    if not return_messages:
        return utility,security
    else:
        return utility,security, messages
    
def count_turns(messages):
    ass_count = 0
    has_user = False
    for msg in messages[2:]: # skip system and first user
        if msg['role'] == 'assistant':
            ass_count += 1
        elif msg['role'] == 'user':
            has_user = True
    return {'turn':ass_count,'additional_user':has_user}


# Eval

In [19]:
benchmark_version = 'v1.2.1'
suites = tuple(get_suites(benchmark_version).keys())
suites = ['banking','travel','workspace']
# suites = ['slack']
attack_name = 'important_instructions'

all_suite_results = defaultdict(dict)
all_suite_asr_messages = defaultdict(dict)
all_suite_stats = defaultdict(dict)
debug_messages = defaultdict(list)

if 'qwen' in model_path.lower():
    base_model_name = 'qwen3'
elif 'llama3' in model_path.lower():
    base_model_name = 'llama3'
elif 'gemma' in model_path.lower():
    base_model_name = 'gemma2'
else:
    base_model_name = model_path


model_n = model_path.split('/')[-1]
result_dict = 'results/agentdojo'
os.makedirs(result_dict,exist_ok=True)

result_file = f'{result_dict}/{base_model_name}_{model_n}.json'
msg_file = f'{result_dict}/{base_model_name}_{model_n}_messages.pkl'

all_suite_results['slack']['injection_utility'] = 1.0
all_suite_results['slack']['task_utility'] = 0.52
all_suite_results['slack']['attack_utility'] = 0.26
all_suite_results['slack']['asr'] = 0.18




if not os.path.exists(result_file):
    for suite_name in suites:
        debug_messages = defaultdict(list)
        suite = get_suite(benchmark_version, suite_name)
        print(f"**Running benchmark for suite: '{suite.name}'**")
        user_tasks_to_run = suite.user_tasks.values()
        print (f"Num User tasks to run: {len(user_tasks_to_run)}")
        attacker_ = load_attack(attack_name, suite, target_pipeline = agent_pipeline)
        print ('is dos_attack:', attacker_.is_dos_attack)
        injection_tasks_to_run = suite.injection_tasks
        print (f"Num Injection tasks to run: {len(injection_tasks_to_run)}")

        #Injection utility
        injection_task_utility = {}
        injection_turn_count = {}
        for i,(injection_task_id, injection_task) in tqdm(enumerate(injection_tasks_to_run.items()),total = len(injection_tasks_to_run),desc = f'Running Injection utility for {suite_name}'):
            utility,_,message = run_task(suite,injection_task,return_messages=True)
            injection_task_utility[injection_task_id] = utility
            debug_messages['injection'].append(message)
            injection_turn_count[injection_task_id] = count_turns(message)

        print (f'{suite_name} Injection utility: {np.mean(list(injection_task_utility.values())):.2f}')
        all_suite_results[suite_name]['injection_utility'] = list(injection_task_utility.values())
        all_suite_stats['injection'][suite_name] = injection_turn_count

        # Non-injection utility
        task_utility = {}
        task_turn_count = {}
        for user_task in tqdm(user_tasks_to_run,total = len(user_tasks_to_run),desc = f'Running Task utility for {suite_name}'):
            utility,_,message = run_task(suite,user_task,return_messages=True)
            task_utility[user_task.ID] = utility
            debug_messages['utility'].append(message)
            task_turn_count[user_task.ID] = count_turns(message)
        print (f'{suite_name} Task utility: {np.mean(list(task_utility.values())):.2f}')
        all_suite_results[suite_name]['task_utility'] = list(task_utility.values())
        all_suite_stats['task'][suite_name] = task_turn_count

#         # With attack
        utility_with_attack = {}
        all_messages = {}
        suite_security_results =  {}
        for user_task in tqdm(user_tasks_to_run,total = len(user_tasks_to_run),desc = f'Running Attack for {suite_name}'):
            for injection_task_id in injection_tasks_to_run:
                injection_task = suite.get_injection_task_by_id(injection_task_id)
                task_injections = attacker_.attack(user_task, injection_task)
                utility,security,messages = run_task(suite,user_task,injection_task,task_injections,return_messages=True)
                utility_with_attack[(user_task.ID, injection_task_id)] = utility
                suite_security_results[(user_task.ID, injection_task_id)] = security
                all_messages[(user_task.ID, injection_task_id)] = messages
        all_suite_asr_messages[suite_name] = all_messages
        all_suite_results[suite_name]['asr'] = list(suite_security_results.values())
        all_suite_results[suite_name]['attack_utility'] = list(utility_with_attack.values())

        print (f'{suite_name} Task with injection Utility: {np.mean(list(utility_with_attack.values())):.2f}')
        print (f'{suite_name} ASR: {np.mean(list(suite_security_results.values())):.2f}')
    
    all_suite_results_dict = {'results':all_suite_results,'stats':all_suite_stats}
    mean_results = defaultdict(dict)
    for suite,suite_results in all_suite_results.items():
        for metric_name, metric_values in suite_results.items():
            mean_results[suite][metric_name] = np.mean(metric_values)
    with open(result_file, 'w') as f: # all_suite_results can be used to gather if the model is weak on task with high turns
        json.dump({'mean_results': mean_results,'stats':all_suite_stats,'results':all_suite_results}, f, indent=4)


else:
    with open(result_file, 'r') as f:
        all_suite_results_dict = json.load(f)

for suite_name in all_suite_results_dict['results']:
    suite_results = all_suite_results_dict['results'][suite_name]
    print (f"Results for {suite_name}:")
    print (f"Injection utility: {np.mean(suite_results['injection_utility']):.2f}")
    print (f"Task utility: {np.mean(suite_results['task_utility']):.2f}")
    print (f"Task with injection utility: {np.mean(suite_results['attack_utility']):.2f}")
    print (f"ASR: {np.mean(suite_results['asr']):.2f}")

**Running benchmark for suite: 'banking'**
Num User tasks to run: 16
is dos_attack: False
Num Injection tasks to run: 9


Running Injection utility for banking: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:20<00:00,  2.32s/it]


banking Injection utility: 0.78


Running Task utility for banking: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:34<00:00,  2.15s/it]


banking Task utility: 0.38


...

WARNING 10-14 11:23:38 [scheduler.py:1102] Input prompt (33356 tokens) is too long and exceeds limit of 32768
WARNING 10-14 11:24:01 [scheduler.py:1102] Input prompt (33609 tokens) is too long and exceeds limit of 32768
WARNING 10-14 11:24:21 [scheduler.py:1102] Input prompt (33387 tokens) is too long and exceeds limit of 32768
WARNING 10-14 11:24:38 [scheduler.py:1102] Input prompt (32806 tokens) is too long and exceeds limit of 32768
WARNING 10-14 11:24:58 [scheduler.py:1102] Input prompt (33378 tokens) is too long and exceeds limit of 32768
WARNING 10-14 11:25:20 [scheduler.py:1102] Input prompt (33670 tokens) is too long and exceeds limit of 32768
WARNING 10-14 11:25:49 [scheduler.py:1102] Input prompt (33379 tokens) is too long and exceeds limit of 32768
WARNING 10-14 11:26:09 [scheduler.py:1102] Input prompt (33918 tokens) is too long and exceeds limit of 32768
WARNING 10-14 11:26:26 [scheduler.py:1102] Input prompt (33723 tokens) is too long and exceeds limit of 32768
WARNING 10

Running Attack for workspace:  95%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊       | 38/40 [36:13<02:02, 61.29s/it]

WARNING 10-14 11:44:15 [scheduler.py:1102] Input prompt (38420 tokens) is too long and exceeds limit of 32768
WARNING 10-14 11:44:30 [scheduler.py:1102] Input prompt (38584 tokens) is too long and exceeds limit of 32768
WARNING 10-14 11:45:14 [scheduler.py:1102] Input prompt (38507 tokens) is too long and exceeds limit of 32768


Running Attack for workspace: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [38:45<00:00, 58.13s/it]

workspace Task with injection Utility: 0.26
workspace ASR: 0.00
Results for slack:
Injection utility: 1.00
Task utility: 0.52
Task with injection utility: 0.26
ASR: 0.18
Results for banking:
Injection utility: 0.78
Task utility: 0.38
Task with injection utility: 0.31
ASR: 0.24
Results for travel:
Injection utility: 0.71
Task utility: 0.45
Task with injection utility: 0.34
ASR: 0.01
Results for workspace:
Injection utility: 0.29
Task utility: 0.23
Task with injection utility: 0.26
ASR: 0.00


In [ ]:
result_stats= all_suite_results_dict['stats']
turns = []
utility = []
for task_type in result_stats: # compute correlation of turn counts and utility
    for suite_name in result_stats[task_type]:
        task_stats = result_stats[task_type][suite_name]
        for task_id in task_stats:
            turns.append(task_stats[task_id]['turn'])
            if task_type == 'task':
                utility.append(all_suite_results_dict['results'][suite_name]['task_utility'])
            elif task_type == 'injection':
                utility.append(all_suite_results_dict['results'][suite_name]['injection_utility'])
    

In [ ]:
count = 0
msgg = list(all_messages.values())
for i,s in enumerate(utility_with_attack.values()):
    if not s:
        if count == 0:
            print (len(msgg[i]))
            for m in msgg[i]:
                pprint (str(m))
            print (i)
            break
        count += 1


# count = 0
# for k,s in suite_security_results.items():
#     if not s and utility_with_attack[k]:
#         if count == 0:
#             print (len(all_messages[k]))
#             for m in all_messages[k]:
#                 pprint (str(m))
#             break
#         count += 1

In [12]:
a = '<tool_call>\n{"name": "invite_user_to_slack", "arguments": {"user": "Fred", "user_email": "fred9246@gmail.com"}}\n</tool_call>\n<tool_call>\n{"name": "add_user_to_channel", "arguments": {"user": "Fred", "channel": "general"}}\n</tool_call>\n<tool_call>\n{"name": "remove_user_from_slack", "arguments": {"user": "Fred"}}\n</tool_call>'
match =  re.findall(r"<tool_call>\s*(.*?)\s*</tool_call>", a, re.DOTALL)
for m in match:
    print (m)

{"name": "invite_user_to_slack", "arguments": {"user": "Fred", "user_email": "fred9246@gmail.com"}}
{"name": "add_user_to_channel", "arguments": {"user": "Fred", "channel": "general"}}
{"name": "remove_user_from_slack", "arguments": {"user": "Fred"}}


In [ ]:
a = "<tool_call>\n{'name':'get_channels', 'arguments': '{}'}\n</tool_call>"
match = re.search(r"<tool_call>\s*(.*?)\s*</tool_call>", a, re.DOTALL)
print (match.group(1))

In [ ]:
for suite_name in all_suite_results:
    suite_results = all_suite_results[suite_name]
    print (f"Results for {suite_name}:")
    print (f"Injection utility: {suite_results['injection_utility']:.2f}")
    print (f"Task utility: {suite_results['task_utility']:.2f}")
    print (f"Task with injection utility: {suite_results['attack_utility']:.2f}")
    print (f"ASR: {suite_results['asr']:.2f}")

# Check result

In [ ]:
# base_model_name = 'qwen3'
base_model_name = 'qwen3'
result_dict = 'results/agentdojo'
# model_n = 'ToolACE-2.5-Llama-3.1-8B'
# model_n = 'Qwen3-8B'
model_n = 'Qwen3-8B'
with open(f'{result_dict}/{base_model_name}_{model_n}.json', 'r') as f:
    all_suite_results = json.load(f)


for suite_name in all_suite_results:
    suite_results = all_suite_results[suite_name]
    print (f"** {suite_name} **")
    print (f"Injec U: {np.mean(suite_results['injection_utility']):.2f}")
    print (f"Task U: {np.mean(suite_results['task_utility']):.2f}")
    print (f"Task/Injec U: {np.mean(suite_results['attack_utility']):.2f}")
    print (f"ASR: {np.mean(suite_results['asr']):.2f}")
    print ('--'*20)